In [28]:
import numpy as np
import xgboost as xgb
import graphviz
import kaggle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import LabelEncoder
import dask.array as da
import dask.dataframe as dd
from dask import compute, persist
from dask.distributed import Client, progress


ImportError: cannot import name 'LazyLoader' from 'xgboost.compat' (C:\Users\alexi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\compat.py)

In [3]:
!kaggle competitions download -c vu-dmt-assigment-2-2023
training_data = dd.read_csv('vu-dmt-assigment-2-2023/training_set_VU_DM.csv')

vu-dmt-assigment-2-2023.zip: Skipping, found more recently modified local copy (use --force to force download)


In [4]:
training_data = training_data.drop(['position', 'gross_bookings_usd', 'date_time'], axis=1)

In [16]:
training_data = training_data.fillna(0)

In [17]:
training_data.dtypes

srch_id                          int64
site_id                          int64
visitor_location_country_id      int64
visitor_hist_starrating        float64
visitor_hist_adr_usd           float64
prop_country_id                  int64
prop_id                          int64
prop_starrating                  int64
prop_review_score              float64
prop_brand_bool                  int64
prop_location_score1           float64
prop_location_score2           float64
prop_log_historical_price      float64
price_usd                      float64
promotion_flag                   int64
srch_destination_id              int64
srch_length_of_stay              int64
srch_booking_window              int64
srch_adults_count                int64
srch_children_count              int64
srch_room_count                  int64
srch_saturday_night_bool         int64
srch_query_affinity_score      float64
orig_destination_distance      float64
random_bool                      int64
comp1_rate               

In [18]:
training_data = training_data[training_data["click_bool"] == 1]

In [19]:
training_data['low_budget'] = (training_data['visitor_hist_adr_usd'] < 117.12)
training_data['medium_budget'] = ((training_data['visitor_hist_adr_usd'] >= 117.12) & (training_data['visitor_hist_adr_usd'] < 200.32))
training_data['high_budget'] = (training_data['visitor_hist_adr_usd'] >= 200.32)

In [20]:
training_data = training_data.astype({'low_budget': int, 'medium_budget': int, 'high_budget': int})

In [21]:
training_data = dd.get_dummies(training_data)

In [22]:
X = training_data.drop('prop_id', axis=1).copy()
X.head()

,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,booking_bool,low_budget,medium_budget,high_budget
12,1,12,187,0.0,0.0,219,2,3.0,1,2.20,...,0.0,0.0,0.0,0.0,11.0,1,1,1,0,0
59,4,5,219,0.0,0.0,219,2,3.0,1,1.61,...,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0
63,6,14,100,0.0,0.0,100,3,4.0,1,1.95,...,0.0,0.0,0.0,0.0,0.0,1,1,1,0,0
68,8,5,219,0.0,0.0,219,3,3.5,1,3.00,...,0.0,0.0,0.0,0.0,0.0,1,1,1,0,0
90,11,5,219,0.0,0.0,219,2,3.5,1,2.20,...,0.0,0.0,0.0,0.0,0.0,1,0,1,0,0


In [23]:
y = training_data['prop_id'].copy()
y.head()

12     68914
59    139893
63    104251
68     27669
90     20499
Name: prop_id, dtype: int64

In [24]:
X_train, X_test = X.random_split([0.9,0.1])
y_train, y_test = y.random_split([0.9,0.1])

In [75]:
# le = LabelEncoder()
# y_train = le.fit_transform(y_train)
# y_test = le.fit_transform(y_test)

In [29]:
import dask_xgboost as dxgb

params ={'objective':'rank:ndcg', 'nround':1000, 'max_depth':16,'eta':0.01,'subsample':0.5,'min_child_weight':1}
cluster = dask.distributed.LocalCluster()
client = dask.distributed.Client(cluster)
bst = dxgb.train(client, params, X_train, y_train)
bst


ImportError: cannot import name 'LazyLoader' from 'xgboost.compat' (C:\Users\alexi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\compat.py)

In [ ]:
clf_xgb = xgb.XGBClassifier(objective='rank:ndcg', missing=None)
model = clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=5, eval_metric='ndcg', eval_set=[(X_test, y_test)])
from sklearn.model_selection import cross_val_predict
predictions = cross_val_predict(model, X_test, y_test, cv=kf)

cm = confusion_matrix(y_test, predictions, labels=clf_xgb.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_xgb.classes_)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\alexi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_16200\4074673400.py", line 2, in <module>
    model = clf_xgb.fit(X_train, y_train, verbose=True, early_stopping_rounds=5, eval_metric='ndcg', eval_set=[(X_test, y_test)])
  File "C:\Users\alexi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\core.py", line 620, in inner_f
    c_str(field),
  File "C:\Users\alexi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\xgboost\sklearn.py", line 1440, in fit
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [    0     1     

2023-05-22 07:39:44,367 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:59360', name: 2, status: running, memory: 0, processing: 0>
2023-05-22 07:39:44,375 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:59310', name: 0, status: running, memory: 0, processing: 0>
2023-05-22 07:39:44,487 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:64387', name: 1, status: running, memory: 0, processing: 0>
2023-05-22 07:39:44,711 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:59360'.
2023-05-22 07:39:44,711 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:59318', name: 2, status: running, memory: 0, processing: 0>
2023-05-22 07:39:44,919 - distributed.scheduler

In [ ]:
test = xgb.DMatrix(X_test.head())
bst.predict(test)

In [97]:
from sklearn.model_selection import cross_val_predict
predictions = cross_val_predict(model, X_test, y_test, cv=kf)
predictions
# cm = confusion_matrix(y_test, predictions, labels=clf_xgb.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf_xgb.classes_)

NameError: name 'model' is not defined